# Community alignment of training questions

In [10]:
import pandas as pd
pd.set_option('display.max_columns', None)
from community_alignment import community_alignment
from ollama_models import ollama_models
models = ollama_models()

In [11]:
dfs = {}
for model in models:
    df = community_alignment(model)
    dfs[model] = df
    df1 = df[['model', 'id', 'title', 'error']].sort_values(by='error')
    display(df1)

Loaded existing index from forecast_index.faiss
Index contains 4883 vectors at initialization


100%|██████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11441.09it/s]

saved community_cogito:latest.json


,model,id,title,error
0,cogito:latest,7811,How many nuclear weapons will be detonated off...,0.8
1,cogito:latest,8734,What percentage of new vehicle production will...,0.8
2,cogito:latest,6633,In how many years will humanity's descendants ...,0.8
3,cogito:latest,1482,What percentage of Americans will be obese or ...,0.8
4,cogito:latest,1454,What will be the total aggregate score given t...,0.8
5,cogito:latest,3054,How many hours will the average American emplo...,0.8
6,cogito:latest,6614,What is the highest value 1 bitcoin will ever ...,0.8
7,cogito:latest,1337,Drake's Equation 1st Parameter R∗: What is the...,0.8
8,cogito:latest,5531,How many seats will be on the Supreme Court of...,0.8
9,cogito:latest,2622,What will be the highest level of annual GDP g...,0.8


Loaded existing index from forecast_index.faiss
Index contains 4883 vectors at initialization


...

,model,id,title,error
0,qwen2.5vl:latest,7811,How many nuclear weapons will be detonated off...,0.8
1,qwen2.5vl:latest,8734,What percentage of new vehicle production will...,0.8
2,qwen2.5vl:latest,6633,In how many years will humanity's descendants ...,0.8
3,qwen2.5vl:latest,1482,What percentage of Americans will be obese or ...,0.8
4,qwen2.5vl:latest,1454,What will be the total aggregate score given t...,0.8
5,qwen2.5vl:latest,3054,How many hours will the average American emplo...,0.8
6,qwen2.5vl:latest,6614,What is the highest value 1 bitcoin will ever ...,0.8
7,qwen2.5vl:latest,1337,Drake's Equation 1st Parameter R∗: What is the...,0.8
8,qwen2.5vl:latest,5531,How many seats will be on the Supreme Court of...,0.8
9,qwen2.5vl:latest,2622,What will be the highest level of annual GDP g...,0.8


Loaded existing index from forecast_index.faiss
Index contains 4883 vectors at initialization


100%|██████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 10768.43it/s]

saved community_qwen3:0.6b-q4_K_M.json


,model,id,title,error
0,qwen3:0.6b-q4_K_M,7811,How many nuclear weapons will be detonated off...,0.8
1,qwen3:0.6b-q4_K_M,8734,What percentage of new vehicle production will...,0.8
2,qwen3:0.6b-q4_K_M,6633,In how many years will humanity's descendants ...,0.8
3,qwen3:0.6b-q4_K_M,1482,What percentage of Americans will be obese or ...,0.8
4,qwen3:0.6b-q4_K_M,1454,What will be the total aggregate score given t...,0.8
5,qwen3:0.6b-q4_K_M,3054,How many hours will the average American emplo...,0.8
6,qwen3:0.6b-q4_K_M,6614,What is the highest value 1 bitcoin will ever ...,0.8
7,qwen3:0.6b-q4_K_M,1337,Drake's Equation 1st Parameter R∗: What is the...,0.8
8,qwen3:0.6b-q4_K_M,5531,How many seats will be on the Supreme Court of...,0.8
9,qwen3:0.6b-q4_K_M,2622,What will be the highest level of annual GDP g...,0.8
